In [5]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from C3D import C3D
                

sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid
from activation_maximization import generate_preferred

In [6]:
os.path.isdir('../cnn_preferred')

True

In [7]:
net = C3D.C3D()
param_file = os.path.join('../model','C3D', 'c3d.pickle')
net.load_state_dict(torch.load(param_file))
net.eval()

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [8]:
net = torchvision.models.alexnet(pretrained=True)

In [9]:
net = torchvision.models.resnet50(pretrained=True)
#net = torchvision.models.vgg16(pretrained=True)

In [10]:
img_mean = np.float32([104, 117, 123])
img_std = np.float32([1,1,1])

# image
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

In [11]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_C3D_TV_norm_all'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [12]:
def hook(module, input, output):
    outputs.append(output.clone())

In [13]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [14]:
layer_list = ['fc8']
layer = layer_list[0]

In [15]:
# initial image for the optimization
fr, h, w = 16, 112,112
initial_video = np.zeros((fr, h,w,3),dtype='float32')

h, w = 224,224
initial_image = np.zeros(( h,w,3),dtype='float32')


initial_input = np.random.randint(0, 256, (h,w,3))

In [16]:
# video

#targetlayer
exec_str_list = ["net."+layer+'.register_forward_hook(hook)']
exec(exec_str_list[0])
exec("num_of_ch = net."+layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_video.transpose(3,0, 1, 2)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

AttributeError: 'ResNet' object has no attribute 'fc8'

In [ ]:
#targetlayer
exec_str_list = ["net.classifier[6].register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = net.classifier[6].weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(1,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_image.transpose(2,0, 1)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [ ]:
#targetlayer
exec_str_list = ["net.conv1.register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = net.conv1.weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(1,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_image.transpose(2,0, 1)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [ ]:
net

In [17]:
target_layer = "net.layer1[2].conv3"
target_layer = "net.layer4[0].bn3"

In [18]:
#targetlayer
exec_str_list = [target_layer +".register_forward_hook(hook)"]
exec(exec_str_list[0])
exec("num_of_ch = "+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(1,num_of_ch,step)
outputs = []
ee = net(torch.Tensor(initial_image.transpose(2,0, 1)[np.newaxis]))
feat_num = outputs[0].detach().numpy().shape

In [19]:
feat_num

(1, 2048, 7, 7)

In [20]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    #'input_size': (16, 112,112,3),
    'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }



In [ ]:
# options for resnet
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 0.01, # learning rate
    'lr_end': 0.01,

    'momentum_start': 0.00001, # gradient with momentum
    'momentum_end': 0.00001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 32,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    #'input_size': (16, 112,112,3),
    'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }



In [21]:
channel_list = [149]

In [25]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    y_index = int(feat_size[2]/2) # the unit in the center of feature map
    x_index = int(feat_size[3]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,y_index,x_index] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=149

iter=1; mean(abs(feat))=0.173477;
iter=2; mean(abs(feat))=0.534218;
iter=3; mean(abs(feat))=0.106737;
iter=4; mean(abs(feat))=0.853883;
iter=5; mean(abs(feat))=0.785848;
iter=6; mean(abs(feat))=0.399554;
iter=7; mean(abs(feat))=0.364635;
iter=8; mean(abs(feat))=0.165315;
iter=9; mean(abs(feat))=0.65821;
iter=10; mean(abs(feat))=1.60475;
iter=11; mean(abs(feat))=0.024184;
iter=12; mean(abs(feat))=1.39661;
iter=13; mean(abs(feat))=1.23307;
iter=14; mean(abs(feat))=0.621662;
iter=15; mean(abs(feat))=1.93355;
iter=16; mean(abs(feat))=1.23472;
iter=17; mean(abs(feat))=0.846034;
iter=18; mean(abs(feat))=1.60672;
iter=19; mean(abs(feat))=1.48101;
iter=20; mean(abs(feat))=0.461665;
iter=21; mean(abs(feat))=1.95339;
iter=22; mean(abs(feat))=0.562983;
iter=23; mean(abs(feat))=1.97458;
iter=24; mean(abs(feat))=1.09955;
iter=25; mean(abs(feat))=1.02402;
iter=26; mean(abs(feat))=1.44887;
iter=27; mean(abs(feat))=1.27376;
iter=28; mean(abs(feat))=1.2373;
iter=29; mean(abs(feat))=1.9159

ValueError: not enough values to unpack (expected 4, got 3)

In [149]:
feature_mask.shape

(1, 2048, 7, 7)